In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import pywt
from mne.time_frequency import tfr_multitaper
from sklearn.model_selection import train_test_split
import torch.cuda as cuda
from torch.autograd import Variable
import math

from utils import create_dataloader,train,bandpower
from dataset import EEG
import wandb
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
# Now do your import
from config import *

In [2]:
# home directory + datasets folder
#C:\Users\Pongk\Desktop\Work\mi-project\EEG-python\dataset\recorded_EEG
path = RECORED_PATH
#path = "/root/EEG_Model/dataset/finetune_EEG/"
#subject to run
left_runs = [3,5,7,9,11]
right_runs = [4,6,8,10,12]
#[3,5,7,9,11]
#[4,6,8,10,12]
#runs = [1,2,3,4,12,13,14,15,16,17]
#runs = [1]
#runs = [7,8,9,10]
subjects = [21]
#recorded eeg class
left_eeg = EEG(path, subjects, left_runs)
raw=left_eeg.data_to_raw()
right_eeg = EEG(path, subjects, right_runs)
right_raw = right_eeg.data_to_raw()

print("Raw done")
data, sf = raw.get_data(), raw.info['sfreq']

print(data.shape)
print(sf)


/Users/pongkornsettasompop/Desktop/work/Motor-Imagery/EEG-python/dataset/recorded_EEG
/Users/pongkornsettasompop/Desktop/work/Motor-Imagery/EEG-python/dataset/recorded_EEG
Raw done
(9, 237454)
250.0


In [3]:
#raw = eeg.set_reference(raw,['CZ'])

In [4]:
#pick channel
# print(raw.info['ch_names'])
raw = left_eeg.pickChannel(raw,['T3','C3','C4','T4','STIM MARKERS'])
right_raw = right_eeg.pickChannel(right_raw,['T3','C3','C4','T4','STIM MARKERS'])
# print(raw.info['ch_names'])
# print(len(raw.info['ch_names']))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [5]:
X_l,y_l = left_eeg.raw_preprocess(raw,event_id=[1.0],rest_stage=False)
#print(X_l.shape,y_l.shape)
X_r,y_r = right_eeg.raw_preprocess(right_raw,event_id=[2.0],rest_stage=False)
X = np.concatenate((X_l,X_r),axis=0)
y = np.concatenate((y_l,y_r),axis=0)
print(X.shape,y.shape)

[853, 5607, 10361, 15114, 19868, 24622, 29375, 34128, 38882, 43635, 48366, 53120, 57873, 62627, 67381, 72134, 76888, 81642, 86396, 91150, 95882, 100623, 105379, 110023, 114773, 119528, 124286, 129039, 133793, 138547, 143279, 148033, 152786, 157539, 162293, 167046, 171800, 176553, 181306, 186059, 190790, 195544, 200297, 205051, 209804, 214558, 219311, 224065, 228819, 233574]
[3230, 7984, 12737, 17490, 22245, 26998, 31751, 36505, 41258, 46012, 50743, 55496, 60251, 65004, 69758, 74511, 79265, 84019, 88773, 93526, 98258, 102984, 107670, 112401, 117154, 121909, 126656, 131416, 136169, 140924, 145656, 150409, 155163, 159916, 164670, 169423, 174176, 178930, 183683, 188436, 193167, 197920, 202674, 207428, 212181, 216934, 221688, 226442, 231198, 235952]
(100, 4, 1750) (100,)


In [6]:
#Feature extraction
bp = bandpower(X, sf, [8, 13])
mul_bp = bandpower(X,sf,[8,13],method='multitaper')
print('Average band power'+ str(bp))
print('Multitaper band'+ str(mul_bp))

Effective window size : 1.024 (s)
Average band power2.5272651796138397
Multitaper band0.1512771642244757


In [7]:
print(X.shape)
print(X[0,0,:])
print(y)

X = left_eeg.apply_baseline(X)
#X = np.transpose(X,(0,2,1))
#print(X[0,0,:])
#print(np.transpose(X,(0,2,1)).shape)

(100, 4, 1750)
[ 3.68788451  3.34832833  2.80377089 ... -1.10874719 -2.28671036
 -3.39157687]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


epochs=eeg.epochs(raw,tmin=-2,tmax=5,baseline=(-2,0))
print(len(epochs.times))
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
#(250*5)
#normal version 
#X = X[:,:,(250*3):]

#new version
#X = X[:,:,(250*8):(250*13)]

print(X.shape)
print(type(X))
#new_X = np.concatenate((fixation_X,imagine_X),axis=2)
print(y)

In [8]:
#Old version use X. New version use new_X

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)
print('Train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)
print(y_train)
print(y_test)


Train size (70, 4, 1750) (70,)
Test size (30, 4, 1750) (30,)
[1 0 1 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 1 1]
[0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0 1 1]


In [ ]:
batch_size = 1750

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)

In [ ]:
#wand setup
#"weightname":"S12_3-9_fir8-14"
#name=f"CNN_S12_3-9_fir8-14",
wandb.login()
wand = wandb.init(
      # Set the project where this run will be logged
      project="Motor-Imagery-New", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"CNN_S21_iir_LFRF_4ch", 
      # Track hyperparameters and run metadata
      # 0.0000001
      config={
      "learning_rate": 0.0000001,
      "architecture": "ConvNet",
      "dataset": "Recorded",
      "epochs": 100000,
      "weightname":"S21_CNN_iir_LFRF_4ch",
      "num_step_per_epoch" : num_step, 
      }
    )

In [ ]:
from model import ConvNet,CNN2D,gamenet,ConvNet2
from torchsummary import summary
from transformer import Transformer
net = ConvNet2()

# device = 'cuda'
# sequence_len=1750 # sequence length of time series
# max_len=5000 # max time series sequence length 
# n_head = 4 # number of attention head
# n_layer = 2 # number of encoder layer
# drop_prob = 0.1
# d_model = 200 # number of dimension ( for positional embedding)
# ffn_hidden = 512 # size of hidden layer before classification 
# in_features = 3 # for univariate time series (1d), it must be adjusted for 1. 
# n_classes = 2
# net =  Transformer( in_features=in_features,
#                      d_model=d_model,
#                      details=False,
#                      n_head=n_head,
#                      max_len=max_len,
#                      seq_len=sequence_len,
#                      ffn_hidden=ffn_hidden,
#                      n_layers=n_layer,
#                      drop_prob=drop_prob,
#                      n_classes=n_classes,
#                      device=device
#                      )
# #summary(net, (2, 641),32)

# Training section

In [ ]:
from utils import train
config = wand.config
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()
device = 'cuda'

train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    loader_train = train_loader,
    loader_test = test_loader,
    vail_loader = None,
    optimizer = optimizer  ,
    criterion = criterion ,
    device = 'cuda',
    wand = wand
)


wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

In [ ]:

import torch
import numpy as np

from torchsummary import summary
from transformer import Transformer
 

sequence_len=1750 # sequence length of time series
max_len=5000 # max time series sequence length 
n_head = 4 # number of attention head
n_layer = 2 # number of encoder layer
drop_prob = 0.1
d_model = 200 # number of dimension ( for positional embedding)
ffn_hidden = 512 # size of hidden layer before classification 
in_features = 3 # for univariate time series (1d), it must be adjusted for 1. 
n_classes = 2
model =  Transformer( in_features=in_features,
                     d_model=d_model,
                     details=False,
                     n_head=n_head,
                     max_len=max_len,
                     seq_len=sequence_len,
                     ffn_hidden=ffn_hidden,
                     n_layers=n_layer,
                     drop_prob=drop_prob,
                     n_classes=n_classes,
                     device=device)

batch_size = 555

#summary(net, (2, 641),32)
#summary(model, input_size=(batch_size,sequence_len,feature))

input_ = torch.from_numpy(np.empty((batch_size,sequence_len,in_features))).float()

model(input_)

print(model)